In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import cudf as cd 
import cuml 
import cupy 

In [3]:
train_data=cd.read_csv("../input/sentiment-analysis/train.csv")

In [4]:
train_data.to_pandas()

In [5]:
train_y=train_data['Sentiment']

In [6]:
import nltk
from nltk.corpus import *
nltk.download('stopwords')
stopwords_list=nltk.corpus.stopwords.words(fileids='english')
print(stopwords_list)

In [7]:
import re
from nltk.stem import WordNetLemmatizer

In [8]:
lem = WordNetLemmatizer()

In [9]:
for i in range(len(train_data)):
#     if (i%1000)==0:
#         print(i)
    sentence=train_data['Phrase'][i]
    text = re.sub("[^a-zA-Z]"," ",sentence)
    text=text.lower()
    words=[]
    final=[]
    stop_word=[]
    words.append(text.split())
    new_words=[]
    for j in range(len(words[0])):
        w=words[0][j]
        if w not in stopwords_list:
            stop_word.append(w)
    final.append(' '.join([lem.lemmatize(word) for word in stop_word]))
    train_data['Phrase'][i]=final

In [10]:
train_data

In [11]:
# output = train_data
# output.to_csv('preprocessed_training_data1.csv', index=False)

In [12]:
# train_data1=pd.read_csv('../input/processed/preprocessed_training_data1.csv')
# train_data1

In [13]:
# train_data1.to_pandas()

In [14]:
train_data1=train_data
train_data1.to_pandas()

In [15]:
train_data1 = train_data1.dropna(axis = 0)
train_data1
index = []
for i in range(len(train_data1)):
    index.append(i)
train_data1.index = index
train_data1

In [31]:
train_data1.to_pandas

In [16]:
import tensorflow

In [17]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential,load_model
from keras.layers import Dense,LSTM,Embedding,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
# list = [0,1,2,3,4]
# plt.bar(list,train_data['Sentiment'].value_counts().sort_index())
k=train_data1['Sentiment'].value_counts().sort_index()

# convert to dataframe
list = []
for i in range(5):
    list.append(k[i])
list


In [19]:
m = [0,1,2,3,4]
plt.bar(m,list)

In [20]:
# plt.hist(list)

In [23]:
df=train_data1['Phrase']
df=df.to_pandas()

In [24]:
tokenizer=Tokenizer(num_words=20000,split=" ")
tokenizer.fit_on_texts(df)
x=tokenizer.texts_to_sequences(df)
x=pad_sequences(x)
x[:5]

In [25]:
x.shape

In [26]:
model=Sequential()
model.add(Embedding(20000,256,input_length=x.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(256,return_sequences=True,dropout=0.3,recurrent_dropout=0.2))
model.add(LSTM(256,dropout=0.3,recurrent_dropout=0.3))
model.add(Dense(5,activation ='softmax'))

In [27]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics='accuracy')
model.summary()

In [35]:
train_data1=train_data1.to_pandas()

In [36]:
y=pd.get_dummies(train_data1['Sentiment']).values
print(type(y))

In [37]:
y.shape

In [38]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [39]:
print(type(x_train))
x_train

In [40]:
print(type(y_train))
y_train

In [41]:
y_train[0]

In [42]:
y_train=y_train.astype('uint32')
y_train

In [ ]:
# a_cpu = np.array(y_train,dtype='uint32')
# y_train.pandas()
# DataFrame.astype(dtype, copy=True, errors=’raise’)
# df1 = pandas.DataFrame(y_train)
# df1
# y_train=astype(uint8)

In [43]:
batch_size=32
epochs=8

model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size,verbose=2)

In [54]:
test_data=pd.read_csv('../input/sentiment-analysis/test.csv')

In [55]:
test_data

In [53]:
for i in range(len(test_data)):
#     if (i%1000)==0:
#         print(i)
    sentence=test_data['Phrase'][i]
    text = re.sub("[^a-zA-Z]"," ",sentence)
    text=text.lower()
    words=[]
    final1=[]
    stop_word=[]
    words.append(text.split())
    new_words=[]
    for j in range(len(words[0])):
        w=words[0][j]
        if w not in stopwords_list:
            stop_word.append(w)
    final1.append(' '.join([lem.lemmatize(word) for word in stop_word]))
    test_data['Phrase'][i]=final1

In [47]:
test_data

In [48]:
test_data = test_data.dropna(axis = 0)
print(test_data)
index = []
for i in range(len(test_data)):
    index.append(i)
test_data.index = index
test_data

In [49]:
df1=test_data['Phrase']
type(df1)

In [50]:
df1

In [ ]:
# df1.to_numpy()

In [ ]:
# type(arr)

In [51]:
tokenizer=Tokenizer(num_words=20000,split=" ")
tokenizer.fit_on_texts(df1)
x1=tokenizer.texts_to_sequences(df1)
x1=pad_sequences(x)
x1[:5]

In [52]:
pred=model.predict(x1)

In [ ]:
# pred[1].max()

In [ ]:
# pred

In [ ]:
index=[]
for i in range(pred.shape[0]):
    max = pred[i].max()
    for j in range(5):
        if pred[i][j]==max:
            index.append(j)

In [ ]:
# index[1]

In [ ]:
#phrase_id = test_data['Phrase_data']
# phrase_id = test_id.to_pandas()

In [ ]:
# arr=np.array(df1)

In [ ]:
# tensorflow.convert_to_tensor(arr,dtype=tensorflow.int32)

In [ ]:
output1 = pd.DataFrame()

In [ ]:
output1['PhraseId'] = test_data['PhraseId']
output1['Sentiment'] = pd.DataFrame(index)

In [ ]:
# output1

In [ ]:
output1.to_csv('final_output.csv', index=False)